In [1]:
import cv2
import numpy as np
import pandas as pd
import time
from pytesseract import image_to_string
import matplotlib.pyplot as plt
import datetime

In [2]:
def euclidean_dist(A,B):
    return np.sqrt(np.sum((A - B)**2))

def manhattan_dist(A,B):
    return np.sum(abs(A - B))

def mse(imageA, imageB):

    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    
    return err

## Shot identification, start time, end time, no of subtopic within topics

In [ ]:
capture = cv2.VideoCapture('test1.mp4')
fps = int(capture.get(cv2.CAP_PROP_FPS))

ret,prev = capture.read()
fnum = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
prev = cv2.cvtColor(prev,cv2.COLOR_BGR2GRAY)
prev_canny = cv2.Canny(prev.copy(),60,120)

flag = True
topic = dict()
ts = -1
subtopic = dict()
header = None

while capture.isOpened() and ret:
    msec = capture.get(cv2.CAP_PROP_POS_MSEC)
    ret1,cur = capture.read()
    fnum = int(capture.get(cv2.CAP_PROP_POS_FRAMES))

    if ret1 == True: 

        cur = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
        cur_canny = cv2.Canny(cur.copy(), 60, 120)

        mean_square_error = mse(cur_canny,prev_canny)
        temp = []
        
        if mean_square_error > 1100:
            
            #thresholding for OCR
            ret,thresh = cv2.threshold(prev,127,255,cv2.THRESH_BINARY)
            
            #frame start time calculation
            start_time = time.strftime('%H:%M:%S', time.gmtime(msec/1000))
            finder = np.sum(prev_canny[0:80,40:940])
            
            if finder < 400000:

                tname = image_to_string(thresh[80:452,10:952])
                
                if tname:
                    #extend end time and subtopic count in dictionary for previous topic
                    l = len(topic)
                    if l:
                        topic[header].extend([start_time, stcount])
                    
                    temp.extend([tname,start_time])
                    ts = 0 # topic
                    stcount = 0
                    #store topic name, start time in dictionary with frame number as key
                    topic[fnum]=temp
                    header = fnum
                    
            elif finder > 800000 and finder < 1500000:
                sname = image_to_string(thresh[0:80,40:940])
                if sname:
                    if ts == 1:
                        #append subtopic name in dictionary with corresponding topic frame number as key
                        subtopic[header].append(sname)
                        stcount += 1
                    elif ts == 0: #new topic,
                        #store subtopic name in dictionary with corresponding topic frame number as key
                        subtopic[header]=[sname]
                        stcount = 1
                        ts = 1
            prev = cur        
            prev_canny = cur_canny
    
    else:
        break
        
capture.release()

#extend end time and subtopic count in dictionary for previous topic
l = len(topic)
if l:
    topic[header].extend([start_time, stcount])

In [4]:
#display topic information
print("\033[1mTopic details are as follows:\033[0m\n")

key = list(topic.keys())
v = list(topic.values())

df = pd.DataFrame(v)
#add column name
df.columns=['Topic Name','St. Time','End time','# Subtopic']
df.insert(0,'Frame #',key)

print(df.to_string(justify='right',index=False,col_space=15))

#display subtopic information
print("\n\033[1mSubtopic details are as follows:\033[0m\n")
for k,v in subtopic.items():
    print(f'  frame number: {k}')
    for i,topic_name in enumerate(v):
        print(f'\t{i+1} {topic_name}')
    print()

Topic details are as follows:

        Frame #                  Topic Name        St. Time        End time      # Subtopic
            229  Distribution of Name Space        00:00:07        00:04:13               6
           7629         DNS in the Internet        00:04:13        00:06:22               5
          11534             Name Resolution        00:06:22        00:11:47              12
          21308                DNS Messages        00:11:47        00:14:11               2

Subtopic details are as follows:

  frame number: 229
	1 Hierarchy of Name Servers
	2 Zones and Domains
	3 What is a Zone?
	4 wrm iemzepe?
	5 The Concept
	6 Zone Transfer

  frame number: 7629
	1 DNS in the Internet
	2 MMitthinlainst
	3 Generic Domains
	4 Country Domains
	5 Inverse Domain

  frame number: 11534
	1 Name Resolution Process
	2 Name Resolution Process (contd...)
	3 Name Resolution Process
	4 Name Resolution Process (contd...)
	5 Name Resolution Process (contd...)
	6 Hierarchy of Name Server

In [ ]:
capture = cv2.VideoCapture('test1.mp4')
fps = int(capture.get(cv2.CAP_PROP_FPS))

ret,prev = capture.read()
fnum = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
prev = cv2.cvtColor(prev,cv2.COLOR_BGR2GRAY)
prev_canny = cv2.Canny(prev.copy(),60,120)

flag = True
topic = dict()
ts = -1
subtopic = dict()
header = None

while capture.isOpened() and ret:
    msec = capture.get(cv2.CAP_PROP_POS_MSEC)
    ret1,cur = capture.read()
    fnum = int(capture.get(cv2.CAP_PROP_POS_FRAMES))

    if ret1 == True: 

        cur = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
        cur_canny = cv2.Canny(cur.copy(), 60, 120)

        mean_square_error = mse(cur_canny,prev_canny)
        temp = []
        
        if mean_square_error > 1100:
            
            #thresholding for OCR
            (thresh1, im_bw) = cv2.threshold(prev, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
            thresh = cv2.threshold(prev, thresh1, 255, cv2.THRESH_BINARY)[1]
            #ret,thresh = cv2.threshold(prev,127,255,cv2.THRESH_BINARY)
            
            #frame start time calculation
            start_time = time.strftime('%H:%M:%S', time.gmtime(msec/1000))
            finder = np.sum(prev_canny[0:80,40:940])
            temp11 = np.sum(thresh[0:80,40:940])
            print('temp11 : {}, finder : {}'.format(temp11,finder))
            if finder < 400000:

                tname = image_to_string(thresh[80:452,10:952])
                
                if tname:
                    #extend end time and subtopic count in dictionary for previous topic
                    l = len(topic)
                    if l:
                        topic[header].extend([start_time, stcount])
                    
                    temp.extend([tname,start_time])
                    ts = 0 # topic
                    stcount = 0
                    #store topic name, start time in dictionary with frame number as key
                    topic[fnum]=temp
                    header = fnum
                    
            elif finder > 800000 and finder < 1500000:
                sname = image_to_string(thresh[0:80,40:940])
                if sname:
                    if ts == 1:
                        #append subtopic name in dictionary with corresponding topic frame number as key
                        subtopic[header].append(sname)
                        stcount += 1
                    elif ts == 0: #new topic,
                        #store subtopic name in dictionary with corresponding topic frame number as key
                        subtopic[header]=[sname]
                        stcount = 1
                        ts = 1
            prev = cur        
            prev_canny = cur_canny
    
    else:
        break
        
capture.release()

#extend end time and subtopic count in dictionary for previous topic
l = len(topic)
if l:
    topic[header].extend([start_time, stcount])

temp11 : 2996250, finder : 1156680
temp11 : 11475000, finder : 244035
temp11 : 3572295, finder : 1154385
temp11 : 3652875, finder : 1413975
temp11 : 2982735, finder : 1638885
temp11 : 3333105, finder : 1260465
temp11 : 3109725, finder : 1159995
temp11 : 2701980, finder : 1295910
temp11 : 2954175, finder : 1086045
temp11 : 3010275, finder : 1118175
temp11 : 11475000, finder : 243780
temp11 : 3267315, finder : 1046775
temp11 : 3301740, finder : 1238535
temp11 : 2886600, finder : 1450950
temp11 : 3199740, finder : 1206405
temp11 : 3197700, finder : 1202070
temp11 : 3095190, finder : 1157700
temp11 : 11475000, finder : 244035
temp11 : 3347895, finder : 1345635
temp11 : 3512370, finder : 1493025
temp11 : 2896290, finder : 1749810
temp11 : 3349680, finder : 1344870
temp11 : 2883540, finder : 1755930
temp11 : 3513900, finder : 1492515
temp11 : 3682200, finder : 2115735
temp11 : 18196545, finder : 1482570
temp11 : 18178185, finder : 1504500
temp11 : 18217710, finder : 1537140
temp11 : 18216180

In [5]:
#display topic information
print("\033[1mTopic details are as follows:\033[0m\n")

key = list(topic.keys())
v = list(topic.values())

df = pd.DataFrame(v)
#add column name
df.columns=['Topic Name','St. Time','End time','# Subtopic']
df.insert(0,'Frame #',key)

print(df.to_string(justify='right',index=False,col_space=15))

#display subtopic information
print("\n\033[1mSubtopic details are as follows:\033[0m\n")
for k,v in subtopic.items():
    print(f'  frame number: {k}')
    for i,topic_name in enumerate(v):
        print(f'\t{i+1} {topic_name}')
    print()

Topic details are as follows:

        Frame #                  Topic Name        St. Time        End time      # Subtopic
            229  Distribution of Name Space        00:00:07        00:04:13               6
           7629         DNS in the Internet        00:04:13        00:06:22               5
          11534             Name Resolution        00:06:22        00:11:47              12
          21308                DNS Messages        00:11:47        00:14:11               2

Subtopic details are as follows:

  frame number: 229
	1 Hierarchy of Name Servers
	2 Zones and Domains
	3 Yee weve
	4 What GanZepe?
	5 The Concept
	6 ree) MEU

  frame number: 7629
	1 PSM mate yates
	2 vss Tg TU LU s
	3 Generic Domains
	4 eC Mey
	5 Inverse Domain

  frame number: 11534
	1 Name Resolution Process
	2 Name Resolution Process (contd...)
	3 Name Resolution Process
	4 Name Resolution Process (contd...)
	5 Name Resolution Process (contd...)
	6 Hierarchy of Name Servers
	7 Recursive Resolution

In [1]:
import cv2

In [12]:

img = cv2.imread('MSE/slide_11.jpg')
img = cv2.rectangle(img,(2,4),(955,66),(0,255,0),2) #subtopic
img = cv2.rectangle(img,(2,74),(956,462),(255,0,0),2) #topic
cv2.imwrite('format_report_slide/format_report_slide_1/Presentation_Image/Topic2.jpg',img)
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img = cv2.rectangle(img,(384,0),(510,128),(0,0,255),13)

In [ ]:
0:80,40:940 4,68 956,462